# Walk the stages of a given ballot

Assumptions:
* Only executing on sequential stages

Logic; taken roughly from [here](https://www.electoral-reform.org.uk/latest-news-and-research/publications/how-to-conduct-an-election-by-the-single-transferable-vote-3rd-edition/#sub-section-18)

For a given ballot
* For each ordered stage
* * if this is the first stage
* * * Establish Quote from (Total Valid Votes / (seats+1)) (Rounded to 2 decimal places)
* * * * If the quota is NOT exact (i.e. x.00)
* * * * * add 0.01 to the quota
* * * If there is a candidate equal to or greater than the quota
* * * * [TRANSFER LOGIC]
* * * [WTF does section 5.1.7b mean?]

* * * Otherwise

## Boring Django Import Stuff

In [1]:
import os
import django
from django.forms.models import model_to_dict
from pprint import pprint

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'voteherder.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from counts.models import *
import pandas as pd


ballot = Ballot.objects.get(id='nia.belfast-east.2017-03-02')
ballot

<Ballot: nia.belfast-east.2017-03-02>

In [2]:
model_to_dict(ballot)

{'id': 'nia.belfast-east.2017-03-02',
 'election': 'nia.2017-03-02',
 'quota': 6727}

In [3]:
candidates = Candidate.objects.filter(standing__in=[ballot])
pprint(list(candidates))

[<Candidate: Chris Lyttle (Alliance - Alliance Party of Northern Ireland)>,
 <Candidate: Naomi Long (Alliance - Alliance Party of Northern Ireland)>,
 <Candidate: Sheila Bodel (Conservative and Unionist Party)>,
 <Candidate: Courtney Robinson (Cross-Community Labour Alternative)>,
 <Candidate: David Douglas (Democratic Unionist Party - D.U.P.)>,
 <Candidate: Joanne Bunting (Democratic Unionist Party - D.U.P.)>,
 <Candidate: Robin Newton (Democratic Unionist Party - D.U.P.)>,
 <Candidate: Georgina Milne (Green Party)>,
 <Candidate: Jordy McKeag (Independent)>,
 <Candidate: John Kyle (Progressive Unionist Party of Northern Ireland)>,
 <Candidate: Séamas de Faoite (SDLP (Social Democratic & Labour Party))>,
 <Candidate: Mairéad O'Donnell (Sinn Féin)>,
 <Candidate: Andrew Girvin (Traditional Unionist Voice - TUV)>,
 <Candidate: Andy Allen (Ulster Unionist Party)>]


In [4]:
stages = Stage.objects.filter(ballot=ballot)
pprint(list(stages))

[<Stage: nia.belfast-east.2017-03-02-1 @ admin>,
 <Stage: nia.belfast-east.2017-03-02-2 @ admin>,
 <Stage: nia.belfast-east.2017-03-02-3 @ admin>,
 <Stage: nia.belfast-east.2017-03-02-4 @ admin>,
 <Stage: nia.belfast-east.2017-03-02-5 @ admin>,
 <Stage: nia.belfast-east.2017-03-02-6 @ admin>,
 <Stage: nia.belfast-east.2017-03-02-7 @ admin>,
 <Stage: nia.belfast-east.2017-03-02-8 @ admin>,
 <Stage: nia.belfast-east.2017-03-02-9 @ admin>,
 <Stage: nia.belfast-east.2017-03-02-10 @ admin>,
 <Stage: nia.belfast-east.2017-03-02-11 @ admin>]


In [16]:
elected = []
excluded = []
for stage in stages:
    # Who are the valid candidates in this stage?
    candidate_counts = StageCell.objects.filter(stage=stage)\
                                .exclude(candidate__in=elected)\
                                .exclude(candidate__in=excluded)\
                                .order_by('-count')
    
    if (c:=candidate_counts.first()).count > ballot.quota:
        print(f'{c.candidate.name} wins in Stage {stage.count_stage} with {c.count} votes')
        elected.append(c.candidate)


Naomi Long wins in Stage 1 with 7610.0 votes


In [17]:
len([_,_])

2

In [10]:
## DO NOT COPY ME, THIS IS NOT REAL CODE
# IT JUST LOOKS LIKE IT ON TV

quota = XXXX
elected = []
excluded = []
seats = 5

while len(elected) < seats:
    candidates = sorted(stage_results)
    
    if candidates.first().vote > quota:
        print("{candidate.first()} Wins!")
        elected.append(candidates.first())
        transferable_votes = candidates.first().vote-quota
        
    else:
        print("{candidate.last()} Wins!")
        excluded.append(candidates.last())
        transferable_votes = candidates.first().vote
        
        stage_results = do_transfer_magic(transferable_votes)
            


{<Candidate: Naomi Long (Alliance - Alliance Party of Northern Ireland)>: 7610.0,
 <Candidate: Joanne Bunting (Democratic Unionist Party - D.U.P.)>: 6007.0,
 <Candidate: Andy Allen (Ulster Unionist Party)>: 5275.0,
 <Candidate: Chris Lyttle (Alliance - Alliance Party of Northern Ireland)>: 5059.0,
 <Candidate: Robin Newton (Democratic Unionist Party - D.U.P.)>: 4729.0,
 <Candidate: David Douglas (Democratic Unionist Party - D.U.P.)>: 4431.0,
 <Candidate: John Kyle (Progressive Unionist Party of Northern Ireland)>: 2658.0,
 <Candidate: Georgina Milne (Green Party)>: 1447.0,
 <Candidate: Mairéad O'Donnell (Sinn Féin)>: 1173.0,
 <Candidate: Andrew Girvin (Traditional Unionist Voice - TUV)>: 917.0,
 <Candidate: Courtney Robinson (Cross-Community Labour Alternative)>: 442.0,
 <Candidate: Sheila Bodel (Conservative and Unionist Party)>: 275.0,
 <Candidate: Séamas de Faoite (SDLP (Social Democratic & Labour Party))>: 250.0,
 <Candidate: Jordy McKeag (Independent)>: 84.0}

NameError: name 'stage' is not defined